*Copyright 2024 The Specials Authors. Licensed under the Apache License, Version 2.0 (the "License").*

_Some of the code in this file is adapted from:_

_modularml/mojo_<br>
_Copyright (c) 2023, Modular Inc._<br>
_Licensed under the Apache License v2.0 with LLVM Exceptions._

# Elementary Functions in Specials

In this Jupyter notebook, we compare the implementations of elementary functions from the Specials package with those found in well-known Mojo and Python packages.

<div class="alert alert-block alert-info">Whenever possible, we parallelize the application of elementary functions to tensors in Mojo. To force sequential execution, set the <code>force_sequential</code> variable to <code>True</code>.</div>

In [1]:
# Option to force sequential execution in Mojo for elementwise operations
alias force_sequential = False


## Table of Contents

- [Experimental Settings](#experimental-settings)
  * [Domains](#domains)
  * [Evaluation Metrics](#evaluation-metrics)
    - [Accuracy](#accuracy)
    - [Runtime Performance](#runtime-performance)
  * [Packages and Auxiliary Functions](#packages-and-auxiliary-functions)
- [Experiment Results](#experiment-results)
  * [Exp](#exp)
  * [Exp2](#exp2)
  * [Expm1](#expm1)
- [Appendix A: System information](#appendix-system-information)

## Experimental Settings

In this section, we outline the experimental settings. From the definition of domains to the metrics used for accuracy and runtime performance evaluation, these settings lay the foundation for an objective assessment.

### Domains

For each elementary function, we uniformly sample 100,000 argument values from intervals of the form $[a_i, b_i]$, referred to as _domains_, where $a_i$ and $b_i$ are the minimum and maximum values of each domain, respectively.

In [2]:
# Sample size
var num_samples = 100_000


We repeat each experiment for single- and double-precision floating-point inputs (`float32` and `float64`).

### Evaluation Metrics

In this section, we present the metrics used for accuracy and runtime performance evaluation.

#### Accuracy

To evaluate the accuracy of function implementations, we measure _error in ulps_. Ulp stands for _unit in the last place_. In this Jupyter notebook, we use the Kahan definition of $\text{ulp}(y)$, where $y$ is an arbitrary real number:

> $\text{ulp}(y)$ is the gap between the two finite floating-point numbers nearest $y$, even if $y$ is not contained in that interval.

Let $\hat{f}$ be an implementation of the mathematical function $f$. Given a representation $\mathbb{T}$ with finite precision (e.g., `float32`) and an input $x \in \mathbb{T}$, the floating-point number $\hat{y} \equiv \hat{f}(x)$ is an approximation of the real number $y \equiv f(x)$. The error in ulps of $\hat{y}$ relative to $y$, $E_{\text{ulps}}(y, \hat{y})$, is given by:

$$E_{\text{ulps}}(y, \hat{y}) = \frac{|y - \hat{y}|}{\text{ulp}(y)}.$$

Ideally, this error is always less than 0.5 in round-to-nearest mode for any pair $(y, \hat{y})$. In fact, this metric has the following interesting property (here we assume that $\mathbb{T}$ is a binary floating-point representation):

> $E_{\text{ulps}}(\hat{y}) < 0.5$ if, and only if, $\hat{y} = \text{RN}(y)$, where $\text{RN}(\cdot)$ is the round-to-nearest function.

The exact but unknown value $y$ is computed with high precision using the Python library [`mpmath`](https://mpmath.org/).

To compare different implementations in terms of accuracy, we calculate the maximum and the root mean square (RMS) of the observed errors for each combination of implementation and domain, with lower values indicating higher accuracy.

#### Runtime Performance

For runtime performance, we measure the execution time using the `benchmark` module in Mojo and a custom function based on the `timeit` module in Python.

We assess runtime performance by calculating the average execution time for each implementation-domain combination, with lower values denoting better performance.

### Packages and Auxiliary Functions

In this section, we import packages and auxiliary functions essential for conducting our experiments and measuring results.

In [3]:
import math

from python import Python
from python.object import PythonObject

Python.add_to_path(".")
Python.add_to_path("..")


In [4]:
%%python

from importlib.util import find_spec
import shutil
import subprocess

fix = """
-------------------------------------------------------------------------
To fix this, follow the steps in the link below:
    https://github.com/modularml/mojo/issues/1085#issuecomment-1771403719
-------------------------------------------------------------------------
"""

def install_if_missing(name: str):
    if find_spec(name.replace("-", "_")):
        return

    print(f"The package `{name}` was not found. We will install it...")
    try:
        if shutil.which("python3"): python = "python3"
        elif shutil.which("python"): python = "python"
        else:
            raise RuntimeError("Python is not on `PATH`. " + fix)
        subprocess.check_call([python, "-m", "pip", "install", name])
    except:
        raise ImportError(f"The package `{name}` was not found. " + fix)

install_if_missing("mpmath")
install_if_missing("numpy")
install_if_missing("tabulate")


In [5]:
%%python

import mpmath as mp
import numpy as np


In [6]:
import specials

from benchmark_utils import run_experiment


## Experiment Results

In this section, we delve into the results of our experiments for each elementary function evaluated.


### Exp

This section shows the experiment results for `exp`, which computes the exponential of `x`.

In [7]:
%%python

def numpy_exp(x):
    """Computes the exponential of a given array using NumPy."""
    return np.exp(x)


def mpmath_exp(x):
    """Computes the exponential of a given array using mpmath."""
    def _mp_exp_impl(a):
        with mp.workdps(50):
            res = mp.exp(mp.mpf(a))
        return mp.nstr(res, 40)

    return np.frompyfunc(_mp_exp_impl, 1, 1)(x)


In [8]:
run_experiment[
    num_domains=5,
    specials_func = specials.exp,
    mojo_func = math.exp,
    dtype = DType.float32,
    force_sequential=force_sequential,
](
    experiment_name="Exp",
    num_samples=num_samples,
    min_values=StaticTuple[5, FloatLiteral](-1e-5, -1.0, -4.0, -20.0, -87.0),
    max_values=StaticTuple[5, FloatLiteral](1e-5, 1.0, 4.0, 20.0, 87.0),
    truth_func=mpmath_exp,
    python_func=numpy_exp,
    python_func_name="NumPy",
)



Experiment: Exp (float32)

                               Maximum Error          RMS Error    Average Execution
Domain        Solution       Observed (ulps)    Observed (ulps)         Time (msecs)
------------  -----------  -----------------  -----------------  -------------------
-1e-05,1e-05  Specials                 0.500              0.288                0.183
              Mojo Stdlib              0.501              0.288                0.072
              NumPy                    1.492              0.568                0.159
------------  -----------  -----------------  -----------------  -------------------
-1,1          Specials                 0.513              0.289                0.187
              Mojo Stdlib              1.523              0.421                0.074
              NumPy                    2.156              0.572                0.147
------------  -----------  -----------------  -----------------  -------------------
-4,4          Specials               

In [9]:
run_experiment[
    num_domains=5,
    specials_func = specials.exp,
    mojo_func = math.exp,
    dtype = DType.float64,
    force_sequential=force_sequential,
](
    experiment_name="Exp",
    num_samples=num_samples,
    min_values=StaticTuple[5, FloatLiteral](-1e-14, -1.0, -8.0, -80.0, -708.0),
    max_values=StaticTuple[5, FloatLiteral](1e-14, 1.0, 8.0, 80.0, 708.0),
    truth_func=mpmath_exp,
    python_func=numpy_exp,
    python_func_name="NumPy",
)



Experiment: Exp (float64)

                               Maximum Error          RMS Error    Average Execution
Domain        Solution       Observed (ulps)    Observed (ulps)         Time (msecs)
------------  -----------  -----------------  -----------------  -------------------
-1e-14,1e-14  Specials                 0.500              0.289                0.388
              Mojo Stdlib              0.500              0.289                0.189
              NumPy                    0.500              0.289                0.722
------------  -----------  -----------------  -----------------  -------------------
-1,1          Specials                 0.514              0.288                0.411
              Mojo Stdlib          2,543.586          1,512.897                0.187
              NumPy                    0.504              0.288                0.721
------------  -----------  -----------------  -----------------  -------------------
-8,8          Specials               

### Exp2

This section shows the experiment results for `exp2`, which computes the base-2 exponential of `x`.

In [10]:
%%python

def numpy_exp2(x):
    """Computes the base-2 exponential of a given array using NumPy."""
    return np.exp2(x)


def mpmath_exp2(x):
    """Computes the base-2 exponential of a given array using mpmath."""
    def _mp_exp2_impl(a):
        with mp.workdps(50):
            res = mp.power(mp.mpf(2), mp.mpf(a))
        return mp.nstr(res, 40)

    return np.frompyfunc(_mp_exp2_impl, 1, 1)(x)


In [11]:
run_experiment[
    num_domains=5,
    specials_func = specials.exp2,
    mojo_func = math.exp2,
    dtype = DType.float32,
    force_sequential=force_sequential,
](
    experiment_name="Exp2",
    num_samples=num_samples,
    min_values=StaticTuple[5, FloatLiteral](-1e-5, -1.0, -5.0, -25.0, -126.0),
    max_values=StaticTuple[5, FloatLiteral](1e-5, 1.0, 5.0, 25.0, 127.0),
    truth_func=mpmath_exp2,
    python_func=numpy_exp2,
    python_func_name="NumPy",
)



Experiment: Exp2 (float32)

                               Maximum Error          RMS Error    Average Execution
Domain        Solution       Observed (ulps)    Observed (ulps)         Time (msecs)
------------  -----------  -----------------  -----------------  -------------------
-1e-05,1e-05  Specials                 0.500              0.289                0.213
              Mojo Stdlib              0.500              0.289                0.059
              NumPy                    0.500              0.289                0.605
------------  -----------  -----------------  -----------------  -------------------
-1,1          Specials                 0.519              0.289                0.204
              Mojo Stdlib            115.862             46.485                0.059
              NumPy                    0.501              0.289                0.636
------------  -----------  -----------------  -----------------  -------------------
-5,5          Specials              

In [12]:
run_experiment[
    num_domains=5,
    specials_func = specials.exp2,
    mojo_func = math.exp2,
    dtype = DType.float64,
    force_sequential=force_sequential,
](
    experiment_name="Exp2",
    num_samples=num_samples,
    min_values=StaticTuple[5, FloatLiteral](-1e-14, -1.0, -10.0, -100.0, -1022.0),
    max_values=StaticTuple[5, FloatLiteral](1e-14, 1.0, 10.0, 100.0, 1023.0),
    truth_func=mpmath_exp2,
    python_func=numpy_exp2,
    python_func_name="NumPy",
)



Experiment: Exp2 (float64)

                               Maximum Error          RMS Error    Average Execution
Domain        Solution       Observed (ulps)    Observed (ulps)         Time (msecs)
------------  -----------  -----------------  -----------------  -------------------
-1e-14,1e-14  Specials                 0.500              0.289                0.398
              Mojo Stdlib              0.500              0.289                0.168
              NumPy                    0.500              0.289                0.731
------------  -----------  -----------------  -----------------  -------------------
-1,1          Specials                 0.517              0.289                0.383
              Mojo Stdlib           6.19e+10           2.50e+10                0.159
              NumPy                    0.507              0.289                0.756
------------  -----------  -----------------  -----------------  -------------------
-10,10        Specials              

### Expm1

This section shows the experiment results for `expm1`, which computes `exp(x) - 1` in a numerically stable way.

In [13]:
%%python

def numpy_expm1(x):
    """Computes `exp(x) - 1` for all elements in the given array using NumPy."""
    return np.expm1(x)


def mpmath_expm1(x):
    """Computes `exp(x) - 1` for all elements in the given array using mpmath."""
    def _mp_expm1_impl(a):
        with mp.workdps(50):
            res = mp.expm1(mp.mpf(a))
        return mp.nstr(res, 40)

    return np.frompyfunc(_mp_expm1_impl, 1, 1)(x)


In [14]:
run_experiment[
    num_domains=5,
    specials_func = specials.expm1,
    mojo_func = math.expm1,
    dtype = DType.float32,
    force_sequential=force_sequential,
](
    experiment_name="Expm1",
    num_samples=num_samples,
    min_values=StaticTuple[5, FloatLiteral](-1e-5, -1.0, -4.0, -20.0, -87.0),
    max_values=StaticTuple[5, FloatLiteral](1e-5, 1.0, 4.0, 20.0, 87.0),
    truth_func=mpmath_expm1,
    python_func=numpy_expm1,
    python_func_name="NumPy",
)



Experiment: Expm1 (float32)

                               Maximum Error          RMS Error    Average Execution
Domain        Solution       Observed (ulps)    Observed (ulps)         Time (msecs)
------------  -----------  -----------------  -----------------  -------------------
-1e-05,1e-05  Specials                 0.500              0.288                0.123
              Mojo Stdlib              0.500              0.288                0.459
              NumPy                    0.500              0.288                0.777
------------  -----------  -----------------  -----------------  -------------------
-1,1          Specials                 0.537              0.289                0.294
              Mojo Stdlib              0.778              0.311                0.653
              NumPy                    0.778              0.311                1.302
------------  -----------  -----------------  -----------------  -------------------
-4,4          Specials             

In [15]:
run_experiment[
    num_domains=5,
    specials_func = specials.expm1,
    mojo_func = math.expm1,
    dtype = DType.float64,
    force_sequential=force_sequential,
](
    experiment_name="Expm1",
    num_samples=num_samples,
    min_values=StaticTuple[5, FloatLiteral](-1e-14, -1.0, -8.0, -80.0, -708.0),
    max_values=StaticTuple[5, FloatLiteral](1e-14, 1.0, 8.0, 80.0, 708.0),
    truth_func=mpmath_expm1,
    python_func=numpy_expm1,
    python_func_name="NumPy",
)



Experiment: Expm1 (float64)

                               Maximum Error          RMS Error    Average Execution
Domain        Solution       Observed (ulps)    Observed (ulps)         Time (msecs)
------------  -----------  -----------------  -----------------  -------------------
-1e-14,1e-14  Specials                 0.500              0.288                0.284
              Mojo Stdlib              0.500              0.288                0.487
              NumPy                    0.500              0.288                0.795
------------  -----------  -----------------  -----------------  -------------------
-1,1          Specials                 0.541              0.289                0.675
              Mojo Stdlib              0.808              0.311                0.718
              NumPy                    0.808              0.311                1.653
------------  -----------  -----------------  -----------------  -------------------
-8,8          Specials             

## Appendix: System information

Below, information about the system used to run the experiment.

In [16]:
%%python

subprocess.run(["modular", "-v"])
subprocess.run(["mojo", "-v"])


modular 0.4.1 (2d8afe15)
mojo 0.7.0 (af002202)


In [17]:
from sys.info import (
    os_is_linux,
    os_is_windows,
    os_is_macos,
    has_sse4,
    has_avx,
    has_avx2,
    has_avx512f,
    has_vnni,
    has_neon,
    is_apple_m1,
    has_intel_amx,
    num_physical_cores,
    _current_target,
    _current_cpu,
    _triple_attr,
)

var os: StringLiteral
if os_is_linux():
    os = "linux"
elif os_is_macos():
    os = "macOS"
else:
    os = "windows"

var cpu = String(_current_cpu())
var arch = String(_triple_attr())

var cpu_features = String("")
if has_sse4():
    cpu_features += " sse4"
if has_avx():
    cpu_features += " avx"
if has_avx2():
    cpu_features += " avx2"
if has_avx512f():
    cpu_features += " avx512f"
if has_vnni():
    if has_avx512f():
        cpu_features += " avx512_vnni"
    else:
        cpu_features += " avx_vnni"
if has_intel_amx():
    cpu_features += " intel_amx"
if has_neon():
    cpu_features += " neon"
if is_apple_m1():
    cpu_features += " apple_m1"

if len(cpu_features) > 0:
    cpu_features = cpu_features[1:]

print("System Information")
print("  OS                   :", os)
print("  CPU                  :", cpu)
print("  Arch                 :", arch)
print("  Num Physical Cores   :", num_physical_cores())
print("  CPU Features         :", cpu_features)


System Information
  OS                   : linux
  CPU                  : haswell
  Arch                 : x86_64-unknown-linux-gnu
  Num Physical Cores   : 4
  CPU Features         : sse4 avx avx2


In [18]:
%%python
import pkg_resources
import sys

def get_version(package):
    """Returns the version of a Python package."""
    return pkg_resources.get_distribution(package).version

print("mpmath version:", mp.__version__)
print("NumPy version:", np.__version__)
print("Python version:", sys.version)
print("Tabulate version:", get_version("tabulate"))


mpmath version: 1.3.0
NumPy version: 1.26.2
Python version: 3.11.5 (main, Sep 11 2023, 14:07:11) [GCC 11.2.0]
Tabulate version: 0.9.0
